# Equivalent stressed volume of a fatigue specimen
### Problem Description
The notched fatigue specimen shown below is subjected to alternating tension / compression.
For the determination of the size factor, the equivalent stressed volume should be evaluated.

<img src="specimen_3d.png" height=450>
<img src="specimen.png" height=450>

The equivalent stresses volume is calculated by:
$$V_{eqv} = \int\limits_{V} \left(\frac{\sigma}{\sigma_{max}}\right)^m dV$$
with $m$ as the weibull modulus. For steel this value is assumed to be 30.<br>
Due to discretisation the integral is transfered into a sum over all elements:
$$V_{eqv} = \sum_{i=1}^n \left(\frac{\sigma_i}{\sigma_{max}}\right)^m V_i$$

Because of the term $\frac{\sigma_i}{\sigma_{max}}$ the magnitude of the loading is arbitrary, as long as defomations are small.

In [ ]:
import os

from pygccx import model as ccx_model
from pygccx import enums
from pygccx import model_keywords as mk
from pygccx import step_keywords as sk
from pygccx.tools import stress_tools as st

import numpy as np

: 

In [ ]:
# change this paths to your location of ccx and cgx
CCX_PATH = os.path.join('../../', 'executables', 'calculix_2.19_4win', 'ccx_static.exe')
CGX_PATH = os.path.join('../../', 'executables', 'calculix_2.19_4win', 'cgx_GLUT.exe')

model = ccx_model.Model(CCX_PATH, CGX_PATH, jobname='specimen_weibull')

### Building the model
The specimen is meshed with C3D20R solid elements. Due to symmetry only 1/8 of the whole specimen is modeled.<br>
The modelling is done in the function build_model.<br>
At first 1/4 of the cross section is modeled in the X-Y plane and then revolved by 90° around the Z-axis.

Only one physical group of the whole volume is defined. So all elements and nodes will be transfered to ccx. The sets for load and boundary conditions are generated later.


To get a better understanding whats going on in build_model, the point-, line-, and surface numbers of the cross section, as well as some important dimensions are shown in the images below.

<img src="cross_section_geo.png" width="600">
<img src="dimensions.png" width="600">


In [ ]:
def build_model(model:ccx_model.Model, l_n:float, d_n:float, d_k:float, r_k:float, order):
    """
    Builds the model geo and mesh using the gmsh API

    Args:
        model (ccx_model.Model)
        l_n (float): total length of specimen
        d_n (float): nominal diameter
        d_k (float): diameter at notch root
        r_k (float): notch radius
        order (int, optional): Element order.
    """

    # calculation of aux variables
    t_k = r_k - (d_n - d_k) / 2
    r_0 = r_k + d_k / 2
    r_1 = (r_k + r_0) / 2
    ang = np.arccos(t_k / r_k)
    ang_2 = np.arccos(t_k / r_1)

    model.clear_gmsh_model()
    gmsh = model.get_gmsh()

    # Add points of cross section in X-Y-plane.
    # most of the time it's more convinent to make a list of 3-tuple
    # and send all points at once to the gmsh-API using a for loop
    pnts = []
    pnts.append((r_0, 0, 0))
    pnts.append((0, 0, 0))
    pnts.append((r_0 - r_1, 0, 0))
    pnts.append((r_0 - r_1 * np.cos(ang / 2), r_1 * np.sin(ang / 2), 0))
    pnts.append((r_0 - r_1 * np.cos(ang_2), r_1 * np.sin(ang_2), 0))
    pnts.append((r_0 - r_k, 0, 0))
    pnts.append((r_0 - r_k * np.cos(ang / 2), r_k * np.sin(ang / 2), 0))
    pnts.append((r_0 - r_k * np.cos(ang), r_k * np.sin(ang), 0))
    pnts.append((0, pnts[3][1], 0))
    pnts.append((0, pnts[4][1], 0))
    pnts.append((0, l_n / 2, 0))
    pnts.append((d_n / 2, l_n / 2, 0))


    for i, p in enumerate(pnts, 1):
        gmsh.model.geo.addPoint(*p, tag=i)
    
    # Add lines
    l1 = gmsh.model.geo.addLine(2,3)
    l2 = gmsh.model.geo.addCircleArc(3, 1, 4)
    l3 = gmsh.model.geo.addCircleArc(4, 1, 5)
    l4 = gmsh.model.geo.addLine(3, 6)
    l5 = gmsh.model.geo.addCircleArc(6, 1, 7)
    l6 = gmsh.model.geo.addCircleArc(7, 1, 8)
    l7 = gmsh.model.geo.addLine(8, 5)
    l8 = gmsh.model.geo.addLine(7, 4)
    l9 = gmsh.model.geo.addLine(4, 9)
    l10 = gmsh.model.geo.addLine(9, 2)
    l11 = gmsh.model.geo.addLine(5, 10)
    l12 = gmsh.model.geo.addLine(10, 9)
    l13 = gmsh.model.geo.addLine(5, 12)
    l14 = gmsh.model.geo.addLine(12, 11)
    l15 = gmsh.model.geo.addLine(11, 10)

    # Add line loops
    c1 = gmsh.model.geo.addCurveLoop([l1,l2,l9,l10])
    c2 = gmsh.model.geo.addCurveLoop([l4,l5,l8,-l2])
    c3 = gmsh.model.geo.addCurveLoop([l6,l7,-l3,-l8])
    c4 = gmsh.model.geo.addCurveLoop([l3,l11,l12,-l9])
    c5 = gmsh.model.geo.addCurveLoop([l13,l14,l15,-l11])

    # Add surfaces
    surfs = [gmsh.model.geo.addPlaneSurface([c]) for c in [c1,c2,c3,c4,c5]]
    # Set all surfaces to transfinite (mapped mesh) and recombine to quads
    for s in surfs:
        gmsh.model.geo.mesh.setTransfiniteSurface(s)
        gmsh.model.geo.mesh.setRecombine(2, s)

    # set element subdevisions
    for l in [l4,l8,l7]:
        gmsh.model.geo.mesh.setTransfiniteCurve(l, 11)
    for l in [l1,l9,l11,l14]:
        gmsh.model.geo.mesh.setTransfiniteCurve(l, 6)
    for l in [l5,l2,l10]:
        gmsh.model.geo.mesh.setTransfiniteCurve(l, 16)
    for l in [l6,l3,l12]:
        gmsh.model.geo.mesh.setTransfiniteCurve(l, 11)
    gmsh.model.geo.mesh.setTransfiniteCurve(l13, 11, coef=1.4)
    gmsh.model.geo.mesh.setTransfiniteCurve(l15, 11, coef=1/1.4)

    # revolve all surfaces by 90° around Z-axis
    revs = [(2,s) for s in surfs]
    out = gmsh.model.geo.revolve(revs, 0,0,0, 0,1,0, angle=np.pi/2, numElements=[20], recombine=True)
    
    # put all volumes in a physical group
    gmsh.model.add_physical_group(3, [1,2,3,4,5], name='SPECIMEN')
    gmsh.model.geo.synchronize()

    # generate mesh
    gmsh.option.setNumber('Mesh.ElementOrder', order)
    gmsh.model.mesh.generate()

In [ ]:
# build model and show in gmsh gui
l_n = 100.0
d_n = 14.0
d_k = 8.0
r_k = 10.0
build_model(model, l_n, d_n, d_k, r_k, order=2)
model.show_gmsh_gui()
model.update_mesh_from_gmsh()

### Boundary conditions and load
The image below shows all applied boundary and load conditions.
A symmetry boundary condition is applied on each cut surface, fixing the dof normal to the face.

The force is applied using a pilot node, located at (0, l_n/2, 0), which is connected to the end face of the specimen by a kinematic coupling .<br>
Only the dof 2 (u_y) is active in the coupling. So the nodes within the coupled face can freely deform in X and Z, but are constrained in Y to the pilot. With this kind of coupling the load is introduced in the model without any stress raisers. <br>
However the rotations of the pilot have to be fixed, to keep the end face perpendicular to the Y-axis.

<img src="boundaries.png">

In [ ]:
# make sets for symmetry boundaries

def isclose(x, value, tol=1e-5):
    """Helper function to compare floats"""
    return abs(x-value) <= tol

# all nodes with x==0, will be fixed in X
nx0 = [nid for nid, c in model.mesh.nodes.items() if isclose(c[0], 0)]
sym_x = model.mesh.add_set('sym_x', enums.ESetTypes.NODE, nx0)
# all nodes with y==0, will be fixed in Y
ny0 = [nid for nid, c in model.mesh.nodes.items() if isclose(c[1], 0)]
sym_y = model.mesh.add_set('sym_y', enums.ESetTypes.NODE, ny0)
# all nodes with z==0, will be fixed in Z
nz0 = [nid for nid, c in model.mesh.nodes.items() if isclose(c[2], 0)]
sym_z = model.mesh.add_set('sym_z', enums.ESetTypes.NODE, nz0)


In [ ]:
# make set for load application
# all nodes with y==ln/2
nload = [nid for nid, c in model.mesh.nodes.items() if isclose(c[1], l_n / 2)]
load = model.mesh.add_set('load', enums.ESetTypes.NODE, nload)

In [ ]:
# add model keywords
elset = model.mesh.get_el_set_by_name('SPECIMEN')
mat = mk.Material('STEEL')
el = mk.Elastic((210000., 0.3))
sos = mk.SolidSection(elset, mat)

# make pilot and kinematic coupling
pilot = model.mesh.add_node((0, l_n/2, 0))
load_surf = model.mesh.add_surface_from_node_set('LOAD', load, enums.ESurfTypes.EL_FACE)
dc = mk.Coupling(enums.ECouplingTypes.KINEMATIC, pilot, load_surf, 'C1', 2)

model.add_model_keywords(
    mk.Boundary(sym_x, 1),
    mk.Boundary(sym_y, 2),
    mk.Boundary(sym_z, 3),
    mk.Boundary(pilot, 1),
    mk.Boundary(pilot, 3,6),
    mat, el, sos,
    dc
)

In [ ]:
# add step and step keywords
step = sk.Step()
step.add_step_keywords(
    sk.Static(),
    sk.Cload(pilot, 2, 1),
    sk.NodeFile([enums.ENodeFileResults.U]),
    sk.ElFile([enums.EElFileResults.S]),
    sk.ElPrint(elset, [enums.EElPrintResults.S, enums.EElPrintResults.EVOL])
)
model.add_steps(step)

In [ ]:
# solve the model and show results in cgx
model.solve(no_cpu=3)
model.show_results_in_cgx()

<img src="stress.png">

In [ ]:
# get result objects from dat file
dat_result = model.get_dat_result()
# result object containing all results in dat file

stress_result = dat_result.get_result_sets_by(entity=enums.EDatEntities.S, step_time=1)[0]
# result set object containing all stress values for time 1.0
# stress_result.values is a dict with key = eid and value = m x n array
# with m = number of int. pnts and n = stress components

volume_result = dat_result.get_result_sets_by(entity=enums.EDatEntities.EVOL, step_time=1)[0]
# result set object containing all volume values for time 1.0
# volume_result.values is a dict with key = eid and value = 1 x 1 array
# containing the volume

In [ ]:
# calculate mises stress for all elements and integration points          
s_mises = {eid: st.get_mises_stress(s) for eid, s in stress_result.values.items()}

# get max mises stress of all int.pnts
s_mises_max = max(np.max(s) for s in s_mises.values())

print('Max. mises stress sig_eqv,max=', s_mises_max)


In [ ]:
# calculate equivalent stressed volume
v_eqv = 0.
for eid, v in volume_result.values.items():
    s = s_mises[eid] # s = vector with len = no int. pnts
    # it is assumed, that the volume for each int. pnt is equal to the 
    # element volume divided by the number of int. pnts
    v_i = v / len(s)
    # equivalent volume for element eid
    v_eqv_e = np.sum((s / s_mises_max)**30 * v_i)
    # add to total
    v_eqv += v_eqv_e

# Equivalent volume has to be multiplied by 8, because only 1/8
# of the specimen was modeled
v_eqv *= 8
print('Equivalent stressed volume V_eqv =', v_eqv)